In [1]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

In [2]:
df=pd.read_csv("../../../jigsaw-toxic-severity-rating/comments_to_score.csv")

In [3]:
df

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...
...,...,...
7532,504235362,"Go away, you annoying vandal."
7533,504235566,This user is a vandal.
7534,504308177,""" \n\nSorry to sound like a pain, but one by f..."
7535,504570375,Well it's pretty fucking irrelevant now I'm un...


In [4]:
from nltk.corpus import stopwords
import re
def text_cleaning(text):
    if text:
        text = ' '.join(text.split('.'))
        text = re.sub('\/',' ',text)
        text = re.sub(r'\\',' ',text)
        text = re.sub(r'((http)\S+)','',text)
        text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z]', ' ', text.strip().lower())).strip()
        text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
        text = [word for word in text.split()]
        return text
    return []

In [5]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [6]:
def lemmatizing(s):
    l=s.split(" ")
    l2=[lemmatizer.lemmatize(word) for word in l]
    s=" ".join(l2)
    return s


In [7]:
df.text = df.text.apply(lambda x: ' '.join(text_cleaning(x)))

In [8]:
df.text = df.text.apply(lambda x: lemmatizing(x))


In [9]:
df

,comment_id,text
0,114890,gjalexei you asked about whether there is an a...
1,732895,look like be have an abuser can you please loo...
2,1139051,i confess to having complete and apparently bl...
3,1434512,freud s idea are certainly much discussed toda...
4,2084821,it is not just you this is a laundry list of s...
...,...,...
7532,504235362,go away you annoying vandal
7533,504235566,this user is a vandal
7534,504308177,sorry to sound like a pain but one by followin...
7535,504570375,well it s pretty fucking irrelevant now i m un...


In [10]:
sent = [row for row in df.text]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

'look like be have an abuser can you please look into this thanks'

In [11]:
sentences = [word_tokenize(sent) for sent in df.text]

In [16]:
allwords=[]
for sentence in sentences:
    for word in sentence:
        allwords.append(word)

In [27]:
phrases = Phrases(sentences, min_count=30, progress_per=10000)

In [28]:
bigram = Phraser(phrases)

In [29]:
sentences2 = bigram[sentences]

In [13]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

In [15]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(3295224, 16589040)

In [16]:
w2v_model.init_sims(replace=True)

C:\Users\Hector\AppData\Local\Temp/ipykernel_8624/514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [18]:
w2v_model.wv.most_similar(positive=["help"])


[('reply', 0.9687471985816956),
 ('feel', 0.9684051871299744),
 ('warned', 0.9671880006790161),
 ('account', 0.9670696258544922),
 ('resolve', 0.9670256972312927),
 ('welcome', 0.9663922786712646),
 ('insult', 0.9645615220069885),
 ('respond', 0.9631102085113525),
 ('blanking', 0.9630785584449768),
 ('asking', 0.9627591371536255)]

In [58]:
w2v_model.wv.most_similar(positive=["shut"])


[('up', 0.9738149642944336),
 ('albino', 0.9042350053787231),
 ('fuck', 0.903282105922699),
 ('niggertard', 0.9000552296638489),
 ('inch', 0.899649441242218),
 ('molesting', 0.8990961909294128),
 ('rw', 0.8983824849128723),
 ('gwernol', 0.8979898691177368),
 ('dvd', 0.8838503360748291),
 ('geek', 0.8692071437835693)]

In [40]:
w2v_model.save("word2vec.model")

In [41]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [21]:
word_vectors = Word2Vec.load("word2vec.model").wv

In [22]:
word_vectors

In [44]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [47]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

[('aswell', 0.9994921684265137),
 ('everyones', 0.9994344711303711),
 ('somone', 0.9993751645088196),
 ('relating', 0.9993699193000793),
 ('compelled', 0.9993450045585632),
 ('smarter', 0.9993321299552917),
 ('dipshit', 0.9992955327033997),
 ('egordon', 0.9992589354515076),
 ('ruined', 0.999235987663269),
 ('ilovedirtbikes', 0.9992008209228516)]

In [48]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('estate', 0.9997875094413757),
 ('menacing', 0.9997500777244568),
 ('portrayed', 0.9997465014457703),
 ('askolnick', 0.9997384548187256),
 ('glen', 0.9997234344482422),
 ('byte', 0.9997187852859497),
 ('san', 0.999714732170105),
 ('commission', 0.9997142553329468),
 ('mandate', 0.9997122883796692),
 ('taliban', 0.9997117519378662)]

In [49]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [24]:
words = pd.DataFrame(word_vectors.index_to_key)


In [25]:
words

,0
0,wikipedia
1,article
2,page
3,fuck
4,like
...,...
9133,portrait
9134,taliban
9135,reptile
9136,enacted


In [29]:
type(w2v_model.wv.most_similar(positive=["retard"]))


list

In [53]:
w2v_model.wv.most_similar(positive=["retard"])

[('inbred', 0.9703599214553833),
 ('twat', 0.9685810804367065),
 ('punk', 0.9682215452194214),
 ('wit', 0.9661067724227905),
 ('ain', 0.9660205841064453),
 ('queer', 0.9658352136611938),
 ('bro', 0.9654372930526733),
 ('bite', 0.9646785855293274),
 ('lame', 0.9645381569862366),
 ('mommy', 0.9640811085700989)]

In [32]:
wordtox={}
for i in w2v_model.wv.most_similar(positive=["retard"]):
    wordtox[i[0]]=i[1]

In [35]:
wordpos={}
for i in w2v_model.wv.most_similar(positive=["thanks"]):
    wordpos[i[0]]=i[1]

In [36]:
words.columns = ['words']

In [42]:
tox_df=pd.DataFrame(wordtox.items())

In [44]:
tox_df.columns=["words", "values"]

In [47]:
pos_df=pd.DataFrame(wordpos.items())

In [48]:
pos_df.columns=["words", "values"]

In [46]:
toxic_score=[]
for i in words["words"]:
    toxic_score.append(tox_df.loc[tox_df["words"] == i, 'values'])

In [49]:
pos_score=[]
for i in words["words"]:
    pos_score.append(pos_df.loc[pos_df["words"] == i, 'values'])

In [50]:
words["tox"]=toxic_score
words["pos"]=pos_score

In [52]:

words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [53]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [54]:
words

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,the,"[0.06806515, 0.07502727, 0.045670383, -0.06687...",1,1,1.595016,1.595016
1,you,"[0.08015253, -0.026329357, 0.012827262, -0.083...",0,-1,1.130235,-1.130235
2,a,"[0.0981175, 0.065807104, 0.07202283, -0.060753...",0,-1,1.778274,-1.778274
3,to,"[0.05603963, 0.01304679, -0.016778206, -0.1138...",0,-1,1.696556,-1.696556
4,i,"[0.08837415, -0.008363632, 0.0071330224, -0.06...",0,-1,1.144578,-1.144578
...,...,...,...,...,...,...
9263,indicative,"[0.08683934, 0.061126996, 0.05128193, -0.10123...",1,1,21.613662,21.613662
9264,shadow,"[0.08866689, 0.055029392, 0.050759654, -0.1061...",0,-1,11.281194,-11.281194
9265,funk,"[0.09127148, 0.054746192, 0.05313637, -0.10553...",0,-1,15.806943,-15.806943
9266,madness,"[0.08765054, 0.061534185, 0.0514894, -0.103769...",1,1,15.600050,15.600050


In [55]:
sentiment_map = words
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display

In [58]:
df.text

0       gjalexei you asked about whether there is an a...
1       look like be have an abuser can you please loo...
2       i confess to having complete and apparently bl...
3       freud s idea are certainly much discussed toda...
4       it is not just you this is a laundry list of s...
                              ...                        
7532                          go away you annoying vandal
7533                                this user is a vandal
7534    sorry to sound like a pain but one by followin...
7535    well it s pretty fucking irrelevant now i m un...
7536    the team name is great britain and northern ir...
Name: text, Length: 7537, dtype: object

In [64]:
sentences3=[]
for i in sentences2:
    sentences3.append(" ".join(i))

In [65]:
df["text"]=sentences3

In [66]:
df.text

0       gjalexei you asked about whether there is an a...
1       look_like be have an abuser can you please loo...
2       i confess to having complete and apparently bl...
3       freud s idea are certainly much discussed toda...
4       it is not just you this is a laundry list of s...
                              ...                        
7532                          go_away you annoying vandal
7533                                this user is a vandal
7534    sorry to sound like a pain but one by followin...
7535    well it s pretty fucking irrelevant now i_m un...
7536    the team name is great britain and northern ir...
Name: text, Length: 7537, dtype: object

In [67]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(), norm=None)
tfidf.fit(df.text)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(df.text)

C:\Users\Hector\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [68]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.text.split()))

In [69]:
%%time
replaced_tfidf_scores = df.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)#this step takes around 3-4 minutes minutes to calculate

Wall time: 1.72 s


In [70]:
replaced_tfidf_scores

0       [9.234564993267135, 9.189343864727395, 5.42790...
1       [5.596978833540749, 2.6622824505731275, 2.4053...
2       [6.868176570656416, 8.318274261392979, 3.30009...
3       [65.0876216367922, 16.773118546938544, 9.86099...
4       [3.8050457038724925, 3.6178183536940915, 2.197...
                              ...                        
7532    [6.40135164921092, 1.5315573107878993, 6.21414...
7533    [2.180115335134195, 3.509347237890869, 1.80890...
7534    [4.654712615263334, 3.3000920259511544, 5.5209...
7535    [3.6586158901208194, 3.8050457038724925, 2.396...
7536    [1.5709225039038672, 6.077564572117022, 4.1346...
Length: 7537, dtype: object

In [71]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [72]:
replaced_closeness_scores = df.text.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

In [73]:
replaced_closeness_scores

0       [0, -1.1302348039775392, -8.0958983888947, -3....
1       [0, -4.958652977928704, -3.7053719759862966, 4...
2       [-1.1445781613963544, -10.01470727091792, -1.6...
3       [4.4474094760884455, 3.275557415500769, -8.948...
4       [-1.8895594747263174, 1.4702613127852697, -1.9...
                              ...                        
7532    [0, -1.1302348039775392, -8.013320632487133, -...
7533    [-1.9141796151404546, -1.4872076875304403, 1.4...
7534    [-3.0670387718934706, -1.6965557751890572, -9....
7535    [-5.8805887997114645, -1.8895594747263174, 3.2...
7536    [1.5950155842943887, 6.193798580597679, -1.935...
Name: text, Length: 7537, dtype: object

In [74]:
df

,comment_id,text
0,114890,gjalexei you asked about whether there is an a...
1,732895,look_like be have an abuser can you please loo...
2,1139051,i confess to having complete and apparently bl...
3,1434512,freud s idea are certainly much discussed toda...
4,2084821,it is not just you this is a laundry list of s...
...,...,...
7532,504235362,go_away you annoying vandal
7533,504235566,this user is a vandal
7534,504308177,sorry to sound like a pain but one by followin...
7535,504570375,well it s pretty fucking irrelevant now i_m un...


In [75]:
df["rate"]=[1 for i in range(len(df))]

In [76]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, df.text, df.rate]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence', 'sentiment']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]

In [77]:
replacement_df.sentiment.value_counts()

1    7537
Name: sentiment, dtype: int64

In [78]:
replacement_df.prediction.value_counts()

0    4130
1    3407
Name: prediction, dtype: int64

In [79]:
replacement_df

,sentiment_coeff,tfidf_scores,sentence,sentiment,sentiment_rate,prediction
0,"[0, -1.1302348039775392, -8.0958983888947, -3....","[9.234564993267135, 9.189343864727395, 5.42790...",gjalexei you asked about whether there is an a...,1,261.932183,1
1,"[0, -4.958652977928704, -3.7053719759862966, 4...","[5.596978833540749, 2.6622824505731275, 2.4053...",look_like be have an abuser can you please loo...,1,-37.533344,0
2,"[-1.1445781613963544, -10.01470727091792, -1.6...","[6.868176570656416, 8.318274261392979, 3.30009...",i confess to having complete and apparently bl...,1,886.624535,1
3,"[4.4474094760884455, 3.275557415500769, -8.948...","[65.0876216367922, 16.773118546938544, 9.86099...",freud s idea are certainly much discussed toda...,1,8468.453356,1
4,"[-1.8895594747263174, 1.4702613127852697, -1.9...","[3.8050457038724925, 3.6178183536940915, 2.197...",it is not just you this is a laundry list of s...,1,281.881128,1
...,...,...,...,...,...,...
7532,"[0, -1.1302348039775392, -8.013320632487133, -...","[6.40135164921092, 1.5315573107878993, 6.21414...",go_away you annoying vandal,1,-90.111189,0
7533,"[-1.9141796151404546, -1.4872076875304403, 1.4...","[2.180115335134195, 3.509347237890869, 1.80890...",this user is a vandal,1,-48.005908,0
7534,"[-3.0670387718934706, -1.6965557751890572, -9....","[4.654712615263334, 3.3000920259511544, 5.5209...",sorry to sound like a pain but one by followin...,1,-707.513645,0
7535,"[-5.8805887997114645, -1.8895594747263174, 3.2...","[3.6586158901208194, 3.8050457038724925, 2.396...",well it s pretty fucking irrelevant now i_m un...,1,-228.010998,0


In [155]:
vd = pd.read_csv("../../../jigsaw-toxic-severity-rating/validation_data.csv")

In [156]:
vd

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"
...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook


In [157]:
vd.less_toxic = vd.less_toxic.apply(lambda x: ' '.join(text_cleaning(x)))
vd.more_toxic = vd.more_toxic.apply(lambda x: ' '.join(text_cleaning(x)))

In [158]:
vd.less_toxic = vd.less_toxic.apply(lambda x: lemmatizing(x))
vd.more_toxic = vd.more_toxic.apply(lambda x: lemmatizing(x))

In [159]:
lt_sentences = [word_tokenize(sent) for sent in vd.less_toxic]

In [160]:
mt_sentences = [word_tokenize(sent) for sent in vd.more_toxic]

In [161]:
lt_sentences2 = bigram[lt_sentences]

In [162]:
mt_sentences2 = bigram[mt_sentences]

In [104]:
lt_sentences3=[]
for i in lt_sentences2:
    lt_sentences3.append(" ".join(i))

In [105]:
mt_sentences3=[]
for i in mt_sentences2:
    mt_sentences3.append(" ".join(i))

In [106]:
vd["less_toxic"]=lt_sentences3

In [107]:
vd["more_toxic"]=mt_sentences3

In [108]:
vd

,worker,less_toxic,more_toxic
0,313,this article suck woo woo wooooooo,what wher is your sexy pic gone from your main...
1,188,and yes people should recognize that but they ...,daphne guinness top of the mornin my favourite...
2,82,western medium yup because every crime in the ...,atom you don t believe actual photo of masturb...
3,347,and you removed it you numbskull i don t care ...,you seem to have sand in your vagina might wan...
4,539,smelly vagina bluerasberry why don t you be a ...,hey way to support nazi you racist
...,...,...,...
30103,461,i m sorry i m not an admin i will give you thr...,get out my large penis
30104,527,i m sorry i m not an admin i will give you thr...,get out my large penis
30105,352,wow are you out of your mind how wa my edit on...,piss off you slant eyed gook
30106,311,wow are you out of your mind how wa my edit on...,piss off you slant eyed gook


In [146]:
print(len(df), len(replacement_df))

7537 7537


In [151]:
replacement_df[replacement_df.sentiment_rate==0]

,sentiment_coeff,tfidf_scores,sentence,sentiment,sentiment_rate,prediction
2236,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[674.1232445085009, 674.1232445085009, 572.923...",lmao_what weak_threat i_havent been_banned yet...,1,0.0,0
2392,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4617.2824966335675, 4617.2824966335675, 4617....",bark_bark bark_bark bark_bark bark_bark bark_b...,1,0.0,0
2921,"[0, 0]","[9.234564993267135, 7.848270632147245]",dominicanyou suck_dick,1,0.0,0
3028,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1194.905616415079, 1310.6611955685898, 1465.6...",fuck_u bitch_fuck u_bitch fuck_u bitch_fuck u_...,1,0.0,0
3132,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1836.452776113066, 1836.452776113066, 1836.45...",bullshit_bullshit bullshit_bullshit bullshit_b...,1,0.0,0
3760,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1979.0048247016605, 1979.0048247016605, 1979....",u_suck u_suck u_suck u_suck u_suck u_suck u_su...,1,0.0,0
4130,"[0, 0]","[8.829099885158971, 9.234564993267135]",penis_penis pensis,1,0.0,0
4174,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4617.2824966335675, 4617.2824966335675, 4617....",shit_shit shit_shit shit_shit shit_shit shit_s...,1,0.0,0
4571,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4617.2824966335675, 4617.2824966335675, 4617....",super_gay super_gay super_gay super_gay super_...,1,0.0,0
4735,"[0, 0, 0]","[7.1551234515873, 7.848270632147245, 8.8290998...",fuck_u bitch_fuck u_bitch,1,0.0,0


In [109]:
replacement_df["sentence"]

0       gjalexei you asked about whether there is an a...
1       look_like be have an abuser can you please loo...
2       i confess to having complete and apparently bl...
3       freud s idea are certainly much discussed toda...
4       it is not just you this is a laundry list of s...
                              ...                        
7532                          go_away you annoying vandal
7533                                this user is a vandal
7534    sorry to sound like a pain but one by followin...
7535    well it s pretty fucking irrelevant now i_m un...
7536    the team name is great britain and northern ir...
Name: sentence, Length: 7537, dtype: object

In [110]:
replacement_df.loc[replacement_df["sentence"] == "article suck woo woo wooooooo", 'sentiment_rate']

Series([], Name: sentiment_rate, dtype: float64)

In [111]:
float(replacement_df.loc[replacement_df["sentence"] == "article suck woo woo wooooooo", 'sentiment_rate'])

TypeError: cannot convert the series to <class 'float'>

In [113]:
less_toxic_score=[]
for i in vd["less_toxic"]:
    less_toxic_score.append(replacement_df.loc[replacement_df["sentence"] == i, 'sentiment_rate'])
            

In [118]:
more_toxic_score=[]
for i in vd["more_toxic"]:
    more_toxic_score.append(replacement_df.loc[replacement_df["sentence"] == i, 'sentiment_rate'])

In [119]:
for i in range(len(less_toxic_score)):
    if len(less_toxic_score[i])==0:
        less_toxic_score[i]=0.0
    else:
        less_toxic_score[i]=less_toxic_score[i].iloc[0]

TypeError: object of type 'numpy.float64' has no len()

In [122]:
for i in range(len(more_toxic_score)):
    if len(more_toxic_score[i])==0:
        more_toxic_score[i]=0.0
    else:
        more_toxic_score[i]=more_toxic_score[i].iloc[0]

In [123]:
vd["less_toxic_score"]=less_toxic_score
vd["more_toxic_score"]=more_toxic_score

In [124]:
vd

,worker,less_toxic,more_toxic,less_toxic_score,more_toxic_score
0,313,this article suck woo woo wooooooo,what wher is your sexy pic gone from your main...,502.455108,65.205077
1,188,and yes people should recognize that but they ...,daphne guinness top of the mornin my favourite...,0.000000,0.000000
2,82,western medium yup because every crime in the ...,atom you don t believe actual photo of masturb...,365.270493,0.000000
3,347,and you removed it you numbskull i don t care ...,you seem to have sand in your vagina might wan...,0.000000,-219.690495
4,539,smelly vagina bluerasberry why don t you be a ...,hey way to support nazi you racist,0.000000,-33.568555
...,...,...,...,...,...
30103,461,i m sorry i m not an admin i will give you thr...,get out my large penis,0.000000,56.762924
30104,527,i m sorry i m not an admin i will give you thr...,get out my large penis,0.000000,56.762924
30105,352,wow are you out of your mind how wa my edit on...,piss off you slant eyed gook,0.000000,399.228578
30106,311,wow are you out of your mind how wa my edit on...,piss off you slant eyed gook,0.000000,399.228578


In [163]:
scores=[]
for index, row in vd.iterrows():
    if row["less_toxic_score"]<row["more_toxic_score"]:
        scores.append(1)
    else:
        scores.append(0)

KeyError: 'less_toxic_score'

In [133]:
vd["scores"]=scores

In [154]:
vd

,worker,less_toxic,more_toxic,less_toxic_score,more_toxic_score,scores
0,313,this article suck woo woo wooooooo,what wher is your sexy pic gone from your main...,502.455108,65.205077,1
24,18,yeah this is pretty lame,fuck you it war now asshole,-120.688666,-12.866763,0
57,382,you think im stupid or smth quit using this ip...,im a fucking cunt,-111.332178,-27.922719,0
59,204,no repsonse just carry on destroying the wiki ...,and this is nothing but a bald faced strawmann...,-172.691531,-47.551315,0
91,268,too cool please please please get a life shoul...,i just re read promise of witch king i had my ...,-543.076493,1961.761926,0
...,...,...,...,...,...,...
30007,170,join the mike garcia brigade together we shall...,pharisee the context in sotah a list of false ...,608.431671,2283.092523,0
30024,733,hello john a you decided to delete my very inf...,daylans a looser and ik kant spel,41.648245,-93.130547,1
30049,465,support stop the pov of nationalist editor,your a fucking faggot you internet thug get of...,77.772212,-65.676708,1
30050,609,support stop the pov of nationalist editor,your a fucking faggot you internet thug get of...,77.772212,-65.676708,1


In [135]:
vd["scores"].value_counts(normalize=True)

0    0.811047
1    0.188953
Name: scores, dtype: float64

In [137]:
vd = vd[vd.less_toxic_score != 0]

In [139]:
vd = vd[vd.more_toxic_score != 0]

In [141]:
vd["scores"].value_counts(normalize=True)

1    0.562442
0    0.437558
Name: scores, dtype: float64

In [142]:
len(vd)

1081